In [1]:
!pip install spacy

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!python -m spacy download de_core_news_sm

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 21.0 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


In [3]:
import os
import glob
import nemo.collections.asr as nemo_asr
import torch
import numpy as np
import ctc_decoders
import torchaudio
import soundfile as sf
import librosa
import pandas as pd

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



In [4]:
from processing.text_processor import preprocess_transcript_for_alignment, preprocess_transcript_for_sentence_split, split_to_sentences
from transcribing.stt_transcribe import get_stt_transcription, get_stt_probs

In [5]:
torch.random.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(torch.__version__)
print(device)

1.11.0
cuda


## Read true transcript
Read the text from the complete protocol provided by the parliament of canton Schwyz. After reading the text process everything and split the whole text into its sentences. 

In [6]:
PATH_TO_TRANSCRIPT = 'data/text_processed_2021-02-24.txt'

In [7]:
with open(PATH_TO_TRANSCRIPT, encoding='utf-8') as f:
    transcript = f.read()
    transcript = preprocess_transcript_for_sentence_split(transcript)
    truth_sentences = split_to_sentences(transcript)

In [8]:
#print(truth_sentences)

## Retrieve all audio files to later match with the corresponding text

In [9]:
PATH_TO_AUDIO = '/data/voice/SZ_parliament/raw/parts'

In [10]:
audio_files = glob.glob(os.path.join(PATH_TO_AUDIO, '*.wav'))
#print(audio_files)

For testing use a test file randomly extracted from one of the audio files

In [11]:
# set a single pregenerated file for testing
TEST_FILE = ['data/processed_0224_part2.wav']

## Transcription
Load and instantiate a pre-trained model from NVIDIA. In this case we use CTC decoding as output. 

In [12]:
# load model once
model = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="stt_de_quartznet15x5")

[NeMo I 2022-06-13 15:24:49 cloud:56] Found existing object /home/user/.cache/torch/NeMo/NeMo_1.8.2/stt_de_quartznet15x5/b82da4f9a238e099c0d94a6a0f7e8b4d/stt_de_quartznet15x5.nemo.
[NeMo I 2022-06-13 15:24:49 cloud:62] Re-using file from: /home/user/.cache/torch/NeMo/NeMo_1.8.2/stt_de_quartznet15x5/b82da4f9a238e099c0d94a6a0f7e8b4d/stt_de_quartznet15x5.nemo
[NeMo I 2022-06-13 15:24:49 common:747] Instantiating model from pre-trained checkpoint


[NeMo W 2022-06-13 15:24:49 modelPT:148] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /raid/noneval.json
    sample_rate: 16000
    labels:
    - ' '
    - a
    - b
    - c
    - d
    - e
    - f
    - g
    - h
    - i
    - j
    - k
    - l
    - m
    - 'n'
    - o
    - p
    - q
    - r
    - s
    - t
    - u
    - v
    - w
    - x
    - 'y'
    - z
    - ä
    - ö
    - ü
    - ß
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    num_workers: 8
    pin_memory: true
    
[NeMo W 2022-06-13 15:24:49 modelPT:155] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    V

[NeMo I 2022-06-13 15:24:49 features:259] PADDING: 16
[NeMo I 2022-06-13 15:24:49 features:276] STFT using torch


[NeMo W 2022-06-13 15:24:50 nemo_logging:349] /home/user/.local/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Torchmetrics v0.9 introduced a new argument class property called `full_state_update` that has
                    not been set for this class (WER). The property determines if `update` by
                    default needs access to the full metric state. If this is not the case, significant speedups can be
                    achieved and we recommend setting this to `False`.
                    We provide an checking function
                    `from torchmetrics.utilities import check_forward_no_full_state`
                    that can be used to check if the `full_state_update=True` (old and potential slower behaviour,
                    default for now) or if `full_state_update=False` can be used safely.
                    
      warnings.warn(*args, **kwargs)
    


[NeMo I 2022-06-13 15:24:51 save_restore_connector:209] Model EncDecCTCModel was successfully restored from /home/user/.cache/torch/NeMo/NeMo_1.8.2/stt_de_quartznet15x5/b82da4f9a238e099c0d94a6a0f7e8b4d/stt_de_quartznet15x5.nemo.


Retrieve the transcript of the given audio sequence

In [13]:
stt_output = get_stt_transcription(model, TEST_FILE)

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

In [14]:
print(stt_output)

ich mich ridrale eire defte päfraktion unterstützt abenfalls schlosse der ersatz und der wiedrigsbau von wonheim phöenix en einsiedler und wihrt die usgaberbewelligung itder höfi vor elf ko nylnulreis millionen francre eichtimigarne danca vemauworde dich friedri drie froaktionssprachendecucimerica aspessim suro buticer ctspräsident ces damem und herre  halte mich auchots deen vorredner gongrum uv tetsas kanton pflichtetic einen uftstiftig könig phoenisenne witerimilforräne des schuckmachtunds witd kommissionssprach zum nivealprojekt zauber es entspricht der vorgabe und derfür davorderige aus bruft um manche mittere ivlirität ihne bedürfnis krachtswerte und darum bitte ec sie mit  amenterra matraf de rgigsfolge und der usgaubewilligung zuestimmeng böcstedank für aufmerksamkeitals wote viel ver fraktionsprachenderalf schmi schiburach jazzepräsident jessy cantonsradin und cantonsrat dephe tsvp fraktion hat grundlagen en dushührige von der gjerigs ruskaber bewilligung für das projekt wurde

For comparison we attach the actual true transcript of the given sequence

In [15]:
#true_transcript = "die zur Verfügung stehenden Mittel fair aufgeteilt werden, um sämtliche Unternehmen unterstützen zu können, die jetzt dringend unsere Hilfe brauchen. Zu erwähnen ist auch, dass bei vielen betroffenen Unternehmen die Personalkosten durch die Kurzarbeitsentschädigung oder durch die Corona-Erwerbsersatzentschädigung seit März 2020 gedeckt"

In [16]:
#print(true_transcript)

In [17]:
!pip install biopython

Defaulting to user installation because normal site-packages is not writeable


## TODO
Implement the start and end point detection for each sentence, then pass it along to the sentence aligner

Get the probabilities for each token in the transcript

In [18]:
def get_stt_probs(model, file):
    logits = model.transcribe(paths2audio_files=file, logprobs=True)[0]
    probs = softmax(logits)
    return probs

def get_stt_logits(model, file):
    logits = model.transcribe(paths2audio_files=file, logprobs=True)[0]
    return logits

def softmax(logits):
    e = np.exp(logits - np.max(logits))
    return e / e.sum(axis=-1).reshape([logits.shape[0], 1])

In [19]:
stt_probs = get_stt_probs(model, TEST_FILE)

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

In [20]:
stt_logits = get_stt_logits(model, TEST_FILE)

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
#print(stt_logits)

## Timestep extraction
Define the duration of a timestep 

In [22]:
# 40ms is duration of a timestep at output of the model
time_stride = 0.02

Get the model's alphabet and replace unknown token with 'space'

In [23]:
labels = list(model.cfg.decoder.vocabulary) + ['blank']
labels[0] = 'space'

In [24]:
print(labels)

['space', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ä', 'ö', 'ü', 'ß', 'blank']


In [25]:
try:
    from plotly import graph_objects as go
except ModuleNotFoundError:
    !pip install plotly
    from plotly import graph_objects as go

In [26]:
'''
# plot probability distribution over characters for each timestep
fig_probs = go.Figure(
    go.Heatmap(z=stt_probs.transpose(),
               colorscale=[
                   [0, 'rgb(30,62,62)'],
                   [1, 'rgb(30,255,30)'],
               ],
               y=labels,
               dx=time_stride,
               name='Probs',
               hovertemplate='Time: %{x:.2f} s<br>Character: %{y}<br>Probability: %{z:.2f}<extra></extra>'),
    layout={
        'height': 300,
        'xaxis': {'title': 'Time, s'},
        'yaxis': {'title': 'Characters'},
        'title': 'Character Probabilities',
        'margin': dict(l=0, r=0, t=40, b=0, pad=0),
    }
)
fig_probs.show()
'''

"\n# plot probability distribution over characters for each timestep\nfig_probs = go.Figure(\n    go.Heatmap(z=stt_probs.transpose(),\n               colorscale=[\n                   [0, 'rgb(30,62,62)'],\n                   [1, 'rgb(30,255,30)'],\n               ],\n               y=labels,\n               dx=time_stride,\n               name='Probs',\n               hovertemplate='Time: %{x:.2f} s<br>Character: %{y}<br>Probability: %{z:.2f}<extra></extra>'),\n    layout={\n        'height': 300,\n        'xaxis': {'title': 'Time, s'},\n        'yaxis': {'title': 'Characters'},\n        'title': 'Character Probabilities',\n        'margin': dict(l=0, r=0, t=40, b=0, pad=0),\n    }\n)\nfig_probs.show()\n"

Extract timestamps for 'space' tokens

In [27]:
#print(np.argmax(stt_probs[6]))

In [28]:
#print(stt_probs[0])

In [29]:
#for i in range(0, stt_probs.shape[0]):
#    print(np.argmax(stt_probs[i]))

In [30]:
# get timestamps for space symbols
spaces = []

state = ''
idx_state = 0

if np.argmax(stt_probs[0]) == 0:
    state = 'space'

for idx in range(1, stt_probs.shape[0]):
    current_char_idx = np.argmax(stt_probs[idx])
    if state == 'space' and current_char_idx != 0 and current_char_idx != 31:
        spaces.append([idx_state, idx-1])
        state = ''
    if state == '':
        if current_char_idx == 0:
            state = 'space'
            idx_state = idx

if state == 'space':
    spaces.append([idx_state, len(stt_probs)-1])

In [31]:
print(labels[31])

blank


In [32]:
no_conf = []
char_argmax = []
char_probs = []

conf_state = ''
index_state = 0

for index in range(1, stt_probs.shape[0]):
    current_char_index = np.argmax(stt_probs[index])
    if conf_state == 'space' and current_char_index != 0 and current_char_index != 31:
        no_conf.append([index_state, index-1])
        conf_state = ''
    if conf_state == '':
        if current_char_index != 31:
            max_index = np.argmax(stt_probs[index])
            char_argmax.append(max_index)
            char_probs.append(stt_probs[index][max_index])
        if current_char_index == 0:
            conf_state = 'space'
            index_state = index


In [33]:
#print(char_probs)

In [34]:
vocab_dict = { i : labels[i] for i in range(0, len(labels), 1)}

In [35]:
print(vocab_dict)

{0: 'space', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 27: 'ä', 28: 'ö', 29: 'ü', 30: 'ß', 31: 'blank'}


In [36]:
char_tokens = list(map(vocab_dict.get, char_argmax))

In [37]:
average_probs = []
current_prob = []
for idx, token in enumerate(char_tokens):
    if token != 'space':
        current_prob.append(char_probs[idx])
    if token == 'space':
        average_probs.append(np.mean(current_prob))
        current_prob = []
        
    

In [38]:
print(len(average_probs))

3520


In [39]:
#print(spaces)

In [40]:
#print(stt_output)

split audio signal into separate words

In [41]:
from IPython.display import Audio, display

In [42]:
#signal, sample_rate = librosa.load(TEST_FILE[0], sr=None)

In [43]:
#print(stt_output)

In [44]:
storage_words = list()
storage_start_time = list()
storage_end_time = list()

In [45]:
# calibration offset for timestamps: 180 ms
offset = -0.18

# split the transcript into words
words = stt_output.split()

# cut words
pos_prev = 0
for j, spot in enumerate(spaces):
    display(words[j])
    pos_end = offset + (spot[0]+spot[1])/2*time_stride
    #display(Audio(signal[int(pos_prev*sample_rate):int(pos_end*sample_rate)],
    #             rate=sample_rate))
    storage_words.append(words[j])
    storage_start_time.append(pos_prev)
    storage_end_time.append(pos_end)
    pos_prev = pos_end

#display(words[j+1])
#display(Audio(signal[int(pos_prev*sample_rate):],
#        rate=sample_rate))


'ich'

'mich'

'ridrale'

'eire'

'defte'

...

'geschlosse'

'messeoder'

'muss'

'sie'

'diskussieren'

In [46]:
print(len(storage_start_time))

3520


In [47]:
processed_stt_output = pd.DataFrame(
    {'startTime': storage_start_time,
     'endTime': storage_end_time,
     'words': storage_words,
     'confidence': average_probs
    })

In [48]:
print(processed_stt_output)

      startTime  endTime         words  confidence
0          0.00     0.12           ich    0.797649
1          0.12     0.36          mich    0.922934
2          0.36     2.21       ridrale    0.809643
3          2.21     2.44          eire    0.649945
4          2.44     2.84         defte    0.915000
...         ...      ...           ...         ...
3515    2326.57  2327.20    geschlosse    0.992727
3516    2327.20  2344.88     messeoder    0.864784
3517    2344.88  2345.07          muss    0.824691
3518    2345.07  2345.24           sie    0.921515
3519    2345.24  2346.26  diskussieren    0.838446

[3520 rows x 4 columns]


## Language Model
Get the 5-gram german language model from huggingface (https://huggingface.co/aware-ai/wav2vec2-xls-r-1b-5gram-german)

In [49]:
#lm_path = "language_model/wav2vec2-xls-r-1b-german-german-5g.binary"

Instantiate beam search with language model

In [50]:
'''
beam_search_lm = nemo_asr.modules.BeamSearchDecoderWithLM(
    vocab=list(model.decoder.vocabulary),
    beam_width=16,
    alpha=2, beta=1.5,
    lm_path=lm_path,
    num_cpus=max(os.cpu_count(), 1),
    input_tensor=False)
'''

'\nbeam_search_lm = nemo_asr.modules.BeamSearchDecoderWithLM(\n    vocab=list(model.decoder.vocabulary),\n    beam_width=16,\n    alpha=2, beta=1.5,\n    lm_path=lm_path,\n    num_cpus=max(os.cpu_count(), 1),\n    input_tensor=False)\n'

In [51]:
#best_sentence_prediction = beam_search_lm.forward(log_probs = np.expand_dims(stt_probs, axis=0), log_probs_length=None)[0][0][1]

In [52]:
#print(best_sentence_prediction)

## Alignment Procedure

In [53]:
from aligning.sentence_aligner import SentenceAligner
from aligning.stt_reader import NeMoSttOutputReader
from aligning.utils import Word

In [54]:
final_stt_output = []
for i in range(0, len(processed_stt_output)):
    final_stt_output.append(Word(storage_words[i], storage_start_time[i], storage_end_time[i], average_probs[i]))

#print(final_stt_output)

In [55]:
sentence_aligner = SentenceAligner()

In [66]:
sentence_alignment, alignment_info = sentence_aligner.predict_one(
    truth_sentences, final_stt_output,
    do_length_ratio_full_transcript_filtering=False,
    do_time_correction=False,
)

In [69]:
print(alignment_info.stt_confidence)

0       0.797649
1       0.797649
2       0.797649
3       0.797649
4       0.797649
          ...   
1632    0.966948
1633    0.946148
1634    0.954532
1635    0.861879
1636    0.864784
Name: stt_confidence, Length: 1637, dtype: float32


In [58]:
NONE_TIME_STRING_REPRESENTATION = '-1.0'
def transform_start_time(time):
    if time is None:
        return NONE_TIME_STRING_REPRESENTATION
    else:
        return time

def transform_end_time(time):
    if time is None:
        return NONE_TIME_STRING_REPRESENTATION
    else:
        return time

In [59]:
with open("final_output_0224.txt", 'w', encoding='utf-8') as f:
        for aligned_sentence in sentence_alignment:
            f.write(f'{transform_start_time(aligned_sentence.start_time)}\t{transform_end_time(aligned_sentence.end_time)}\t{aligned_sentence.sentence}\n')

## IOU Estimation
With the following features the IOU is estimated:
- Length Ratio
- Alignment Score
- Mean speech recognition confidence
- Chars per second

Given the IOU a threshold is set determining whether the sentence is discarded for training or not. 

In [82]:
length_ratio = alignment_info.truth_length / alignment_info.stt_length
chars_per_second = alignment_info.truth_length / alignment_info.audio_duration

features = pd.DataFrame(
                        {'confidence': alignment_info.stt_confidence,
                        'score': alignment_info.score,
                        'length_ratio': length_ratio,
                        'chars_per_second': chars_per_second
                        }
                       )
features['chars_per_second'].replace(np.inf, 0, inplace=True)
print(features)

      confidence       score  length_ratio  chars_per_second
0       0.797649  -47.185579     60.000000       1500.000000
1       0.797649  -17.876490     21.333333          0.000000
2       0.797649  -32.278370     40.333333          0.000000
3       0.797649   -9.285895     10.000000          0.000000
4       0.797649  -14.844515     17.333333          0.000000
...          ...         ...           ...               ...
1632    0.966948  -16.485623      0.847222         12.007874
1633    0.946148  -32.477824      1.129870         18.510638
1634    0.954532 -113.817349      1.084071         14.488468
1635    0.861879  -47.972506      1.340426         20.160000
1636    0.864784   -4.751816      0.666667          0.339367

[1637 rows x 4 columns]


## Audio Splitting
Given the sequences determined by the alignments with sentences, we split up the large audio files into digestable sequences. 
To prepare the sequences for training manifests for NeMo will be built. 